In [4]:
# Show tables in the current database

spark.sql("SHOW TABLES").show()

+---------+----------------+-----------+
|namespace|       tableName|isTemporary|
+---------+----------------+-----------+
|  default|teststocksymbols|      false|
+---------+----------------+-----------+



In [55]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("TestStockSymbols") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Define the new schema for the table
schema = "symbol STRING, name STRING"

# Create a DataFrame with sample data
data = [
    ("AAPL", "Apple Inc."),
    ("MSFT", "Microsoft Corp."),
    ("GOOGL", "Alphabet Inc."),
    ("TSLA", "Tesla Inc."),
    ("AMZN", "Amazon.com Inc."),
    ("NVDA", "NVIDIA Corporation")
]

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)

# Write DataFrame to Delta table in HDFS
df.write.format("delta").mode("overwrite").save("hdfs://namenode:8020/testlakehouse/TestStockSymbols")

# Drop the existing table if it exists
spark.sql("DROP TABLE IF EXISTS TestStockSymbols")

# Register the updated Delta table in Spark SQL catalog
spark.sql("""
    CREATE TABLE TestStockSymbols
    USING DELTA
    LOCATION 'hdfs://namenode:8020/testlakehouse/TestStockSymbols'
""")

# Query to select all records from the test_table
result = spark.sql("SELECT * FROM teststocksymbols")

# Show the results
result.show()

24/08/12 22:46:07 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `default`.`teststocksymbols` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
+------+------------------+
|symbol|              name|
+------+------------------+
|  NVDA|NVIDIA Corporation|
|  MSFT|   Microsoft Corp.|
|  AMZN|   Amazon.com Inc.|
| GOOGL|     Alphabet Inc.|
|  TSLA|        Tesla Inc.|
|  AAPL|        Apple Inc.|
+------+------------------+



In [56]:
# Show tables in the current database

spark.sql("SHOW TABLES").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|teststockcurrentp...|      false|
|  default|    teststocksymbols|      false|
|  default|teststocktransaction|      false|
|         |     stock_news_html|      false|
+---------+--------------------+-----------+



In [12]:
current_database = spark.sql("SELECT current_database()").collect()[0][0]
print(f"Current database: {current_database}")


Current database: default


In [ ]:
!pip install psycopg2-binary

In [46]:
import psycopg2
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder \
    .appName("TestStockCurrentPrice") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Define the schema for the table
schema = "symbol STRING, price DOUBLE, volume LONG"

# Sample data to create the DataFrame
data = [
    ("AAPL", 234.82, 43234280),
    ("MSFT", 193.02, 33994710),
    ("GOOGL",185.5, 12760100),
    ("NVDA", 449.52, 17175680),
    ("AMZN", 126.36, 214769500),
    ("TSLA", 256.56, 126332500)
]

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)

# Define the path for the Delta table
delta_path = "hdfs://namenode:8020/testlakehouse/TestStockCurrentPrice"

# Drop the existing Delta table if it exists
spark.sql(f"DROP TABLE IF EXISTS TestStockCurrentPrice")

# Write DataFrame to Delta table in HDFS
df.write.format("delta").mode("overwrite").save(delta_path)

# Create the Delta table in Spark SQL catalog
spark.sql(f"""
    CREATE TABLE TestStockCurrentPrice
    USING DELTA
    LOCATION '{delta_path}'
""")

print("Table TestStockCurrentPrice created and registered successfully.")

# Query some data
data_preview = spark.sql("SELECT * FROM TestStockCurrentPrice LIMIT 6")
data_preview.show(truncate=False)

24/08/12 22:30:06 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `default`.`teststockcurrentprice` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
Table TestStockCurrentPrice created and registered successfully.
+------+------+---------+
|symbol|price |volume   |
+------+------+---------+
|GOOGL |185.5 |12760100 |
|MSFT  |193.02|33994710 |
|AMZN  |126.36|214769500|
|NVDA  |449.52|17175680 |
|TSLA  |256.56|126332500|
|AAPL  |234.82|43234280 |
+------+------+---------+



In [41]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CreateTestStockTransaction") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Define the schema for the DataFrame
schema = """
    AAPL_price DOUBLE,
    AAPL_volume LONG,
    AMZN_price DOUBLE,
    AMZN_volume LONG,
    GOOG_price DOUBLE,
    GOOG_volume LONG,
    MSFT_price DOUBLE,
    MSFT_volume LONG,
    NVDA_price DOUBLE,
    NVDA_volume LONG,
    TSLA_price DOUBLE,
    TSLA_volume LONG
"""

# Sample data to create the DataFrame
data = [
    (234.82, 43234280, 126.36, 214769500, 185.5, 12760100, 193.02, 33994710, 449.52, 17175680, 256.56, 126332500)
]

# Create DataFrame with the sample data
df = spark.createDataFrame(data, schema=schema)

# Add the timestamp column
df_with_timestamp = df.withColumn("timestamp", F.current_timestamp())

# Define the path for the Delta table
delta_path = "hdfs://namenode:8020/testlakehouse/TestStockTransaction"

# Write DataFrame to Delta table in HDFS
df_with_timestamp.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(delta_path)

# Drop the existing table if it exists
spark.sql("DROP TABLE IF EXISTS TestStockTransaction")

# Register the Delta table in Spark SQL catalog
spark.sql(f"""
    CREATE TABLE TestStockTransaction
    USING DELTA
    LOCATION '{delta_path}'
""")

print("Delta table TestStockTransaction created and registered successfully.")

# Query some data to confirm
data_preview = spark.sql("SELECT * FROM TestStockTransaction LIMIT 6")
data_preview.show(truncate=False)


24/08/12 22:27:05 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `default`.`teststocktransaction` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
Delta table TestStockTransaction created and registered successfully.
+----------+-----------+----------+-----------+----------+-----------+----------+-----------+----------+-----------+----------+-----------+--------------------------+
|AAPL_price|AAPL_volume|AMZN_price|AMZN_volume|GOOG_price|GOOG_volume|MSFT_price|MSFT_volume|NVDA_price|NVDA_volume|TSLA_price|TSLA_volume|timestamp                 |
+----------+-----------+----------+-----------+----------+-----------+----------+-----------+----------+-----------+----------+-----------+--------------------------+
|234.82    |43234280   |126.36    |214769500  |185.5     |12760100   |193.02    |33994710   |449.52    |17175680   |256.56    |126332500  |2024-08-12 22:27:03.951914|


In [ ]:
import psycopg2
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp

# Initialize Spark session
spark = SparkSession.builder \
    .appName("AppendDataToTestStockTransaction") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Define the Delta path for TestStockTransaction
delta_path_TestStockTransaction = "hdfs://namenode:8020/testlakehouse/TestStockTransaction"

# Connect to PostgreSQL and fetch a random row
try:
    conn = psycopg2.connect(
        dbname="bqnbcj8kxuogsigyhnzy",
        user="ufjklpchveyybgraqhxu",
        password="AyR5dzFuySPaAcWd5po1AJMK063nkG",
        host="bqnbcj8kxuogsigyhnzy-postgresql.services.clever-cloud.com",
        port="50013"
    )
    cur = conn.cursor()

    # Fetch a random row from PostgreSQL
    cur.execute("""
        SELECT * FROM sample_stock_data ORDER BY RANDOM() LIMIT 1
    """)

    # Fetch the row
    random_row = cur.fetchone()

finally:
    # Close the cursor and connection
    cur.close()
    conn.close()

# Define the columns
columns = [
    'AAPL_price', 'AAPL_volume', 'AMZN_price', 'AMZN_volume', 
    'GOOG_price', 'GOOG_volume', 'MSFT_price', 'MSFT_volume',
    'NVDA_price', 'NVDA_volume', 'TSLA_price', 'TSLA_volume'
]

# Convert the row to a DataFrame
row_df = pd.DataFrame([random_row], columns=columns)

# Convert to Spark DataFrame
row_spark_df = spark.createDataFrame(row_df)

# Convert the data types
row_spark_df_corrected = row_spark_df \
    .withColumn("AAPL_price", col("AAPL_price").cast("double")) \
    .withColumn("AAPL_volume", col("AAPL_volume").cast("long")) \
    .withColumn("AMZN_price", col("AMZN_price").cast("double")) \
    .withColumn("AMZN_volume", col("AMZN_volume").cast("long")) \
    .withColumn("GOOG_price", col("GOOG_price").cast("double")) \
    .withColumn("GOOG_volume", col("GOOG_volume").cast("long")) \
    .withColumn("MSFT_price", col("MSFT_price").cast("double")) \
    .withColumn("MSFT_volume", col("MSFT_volume").cast("long")) \
    .withColumn("NVDA_price", col("NVDA_price").cast("double")) \
    .withColumn("NVDA_volume", col("NVDA_volume").cast("long")) \
    .withColumn("TSLA_price", col("TSLA_price").cast("double")) \
    .withColumn("TSLA_volume", col("TSLA_volume").cast("long"))

# Add the timestamp column
row_spark_df_with_timestamp = row_spark_df_corrected.withColumn("timestamp", current_timestamp())

# Append data to the TestStockTransaction Delta table
row_spark_df_with_timestamp.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .save(delta_path_TestStockTransaction)

print("Data appended to TestStockTransaction successfully.")

# Query some data to confirm
data_preview = spark.sql("SELECT * FROM TestStockTransaction LIMIT 6")
data_preview.show(truncate=False)

###########################################################################################################################################

# Define the path for the Delta table
delta_path_TestStockCurrentPrice = "hdfs://namenode:8020/testlakehouse/TestStockCurrentPrice"


# Transform the DataFrame from wide to long format
melted_df = row_spark_df.selectExpr(
    "stack(6, 'AAPL', AAPL_price, 'AMZN', AMZN_price, 'GOOG', GOOG_price, 'MSFT', MSFT_price, 'NVDA', NVDA_price, 'TSLA', TSLA_price) as (symbol, price)"
).join(
    row_spark_df.selectExpr(
        "stack(6, 'AAPL', AAPL_volume, 'AMZN', AMZN_volume, 'GOOG', GOOG_volume, 'MSFT', MSFT_volume, 'NVDA', NVDA_volume, 'TSLA', TSLA_volume) as (symbol, volume)"
    ),
    on="symbol"
).select("symbol", "price", "volume")

# Convert columns to the appropriate data types
melted_df_corrected = melted_df \
    .withColumn("price", col("price").cast("double")) \
    .withColumn("volume", col("volume").cast("long"))

# Write the corrected DataFrame to the Delta table
melted_df_corrected.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(delta_path_TestStockCurrentPrice)

print("Delta table updated TestStockCurrentPrice successfully.")
# Query some data to confirm
data_preview = spark.sql("SELECT * FROM TestStockCurrentPrice LIMIT 6")
data_preview.show(truncate=False)

: 

In [50]:
# Show the table schema
schema = spark.sql("DESCRIBE TestStockCurrentPrice")
schema.show(truncate=False)

# Count the number of rows in the table
row_count = spark.sql("SELECT COUNT(*) AS record_count FROM TestStockCurrentPrice")
row_count.show(truncate=False)

# Query some data
data_preview = spark.sql("SELECT * FROM TestStockCurrentPrice LIMIT 6")
data_preview.show(truncate=False)


+---------------+---------+-------+
|col_name       |data_type|comment|
+---------------+---------+-------+
|symbol         |string   |       |
|price          |double   |       |
|volume         |bigint   |       |
|               |         |       |
|# Partitioning |         |       |
|Not partitioned|         |       |
+---------------+---------+-------+

+------------+
|record_count|
+------------+
|6           |
+------------+

+------+------+---------+
|symbol|price |volume   |
+------+------+---------+
|AAPL  |209.27|119548600|
|AMZN  |161.02|83149440 |
|GOOG  |160.64|34907820 |
|MSFT  |395.15|40709240 |
|NVDA  |100.45|552842400|
|TSLA  |198.88|100308800|
+------+------+---------+



In [14]:
import requests
from bs4 import BeautifulSoup

# Make a request to the URL
url = 'https://seekingalpha.com/market-news'
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the text from the entire HTML
text = soup.get_text()

# List of keywords to count
keywords = ['AAPL', 'AMZN', 'GOOG', 'MSFT', 'NVDA', 'TSLA']

# Count occurrences of each keyword
keyword_counts = {keyword: text.upper().count(keyword) for keyword in keywords}

print("Count the key words appear on the news paper:",keyword_counts)

Count the key words appear on the news paper: {'AAPL': 0, 'AMZN': 0, 'GOOG': 0, 'MSFT': 0, 'NVDA': 0, 'TSLA': 0}


In [26]:
import requests
from bs4 import BeautifulSoup
from kafka import KafkaProducer
from datetime import datetime

# Make a request to the URL
url = 'https://seekingalpha.com/market-news'
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the text from the entire HTML
text = soup.get_text()

# Kafka configuration
bootstrap_servers = '172.18.0.99:9092'
topic = 'phongdinhcs-test-topic'

# Create a Kafka producer
producer = KafkaProducer(
    bootstrap_servers=bootstrap_servers,
    value_serializer=lambda v: str(v).encode('utf-8')  # Convert messages to bytes
)

# Send the HTML text to the Kafka topic
producer.send(topic, value=text)

# Wait for all messages to be sent
producer.flush()

# Get current date and time
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

print(f"HTML text pushed to Kafka topic 'phongdinhcs-test-topic' at {now}")


HTML text pushed to Kafka topic 'phongdinhcs-test-topic' at 2024-08-12 20:27:22


In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring, to_json, struct
import json

# Initialize Spark session
spark = SparkSession.builder \
    .appName("ReadDeltaTable") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Path to the Delta table in HDFS
delta_table_path = "hdfs://namenode:8020/testlakehouse/stock_news_html"

# Read the Delta table into a DataFrame
df = spark.read.format("delta").load(delta_table_path)

# Determine the last column name
last_column_name = df.columns[-1]

# Truncate the last column to characters 2000 to 3000
truncated_df = df.select(
    *[
        (substring(col(column_name), 2000, 3000).alias(column_name) if column_name == last_column_name else col(column_name))
        for column_name in df.columns
    ]
)

# Convert the DataFrame to JSON format
json_df = truncated_df.select(to_json(struct(*truncated_df.columns)).alias("json"))

# Collect the JSON data
json_data = json_df.collect()

# Print the JSON data in a readable format
for row in json_data:
    print(json.dumps(json.loads(row["json"]), indent=4))

# Stop Spark session
spark.stop()


{
    "timestamp": "2024-08-12 00:49:23",
    "AAPL": 0,
    "AMZN": 0,
    "GOOG": 2,
    "MSFT": 0,
    "NVDA": 0,
    "TSLA": 0,
    "hdfs_location": "hdfs://namenode:8020/testlakehouse/stock_news_html",
    "crawl_from_website": "\"https://seekingalpha.com/market-news\"",
    "html_text": "' surpasses $1B worldwideAMC -1.57%Yesterday, 5:20 PM15 CommentsNew York City, Dallas lead apartment construction boom in 2024 - reportEQR +1.36%Yesterday, 4:18 PM2 CommentsGoldman screens for post-selloff stocks with healthy fundamentals at a discountIWB +0.47%Yesterday, 4:04 PM15 CommentsSA Asks: Will the Fed do an emergency rate cut?AGG +0.35%Yesterday, 3:29 PM59 CommentsTrump says Microsoft told him Iran was behind campaign hackDJT -0.49%Yesterday, 1:11 PM175 CommentsDisney unveils 'Avatar', 'Indiana Jones', 'Monsters, Inc' and 'Cars' attractions in big parks and resort pushDIS +0.29%Yesterday, 11:04 AM7 CommentsMarketWise founder Porter Stansberry resigns as CEO, ChairmanMKTW +5.88%Yesterday

In [51]:
# Create a temporary view for the Delta table
spark.sql("""
    CREATE OR REPLACE TEMP VIEW stock_news_html AS
    SELECT * FROM delta.`hdfs://namenode:8020/testlakehouse/stock_news_html`
""")

# Show all tables in the current database
tables = spark.sql("SHOW TABLES").select("tableName").rdd.flatMap(lambda x: x).collect()

# Describe each table and show the record count
for table in tables:
    print(f"Describing table: {table}")
    
    # Describe the table
    description = spark.sql(f"DESCRIBE {table}")
    description.show(truncate=False)
    
    # Count the number of records in the table
    record_count = spark.sql(f"SELECT COUNT(*) AS record_count FROM {table}")
    record_count.show(truncate=False)

Describing table: teststockcurrentprice
+---------------+---------+-------+
|col_name       |data_type|comment|
+---------------+---------+-------+
|symbol         |string   |       |
|price          |double   |       |
|volume         |bigint   |       |
|               |         |       |
|# Partitioning |         |       |
|Not partitioned|         |       |
+---------------+---------+-------+

+------------+
|record_count|
+------------+
|6           |
+------------+

Describing table: teststocksymbols
+---------------+---------+-------+
|col_name       |data_type|comment|
+---------------+---------+-------+
|symbol         |string   |       |
|name           |string   |       |
|               |         |       |
|# Partitioning |         |       |
|Not partitioned|         |       |
+---------------+---------+-------+

+------------+
|record_count|
+------------+
|3           |
+------------+

Describing table: teststocktransaction
+---------------+---------+-------+
|col_name   

In [58]:
# Show all tables in the current database
tables = spark.sql("SHOW TABLES").select("tableName").rdd.flatMap(lambda x: x).collect()

# Describe each table
for table in tables:
    print(f"Describing table: {table}")
    description = spark.sql(f"DESCRIBE {table}")
    description.show(truncate=False)

Describing table: teststockcurrentprice
+---------------+---------+-------+
|col_name       |data_type|comment|
+---------------+---------+-------+
|symbol         |string   |       |
|price          |double   |       |
|volume         |bigint   |       |
|               |         |       |
|# Partitioning |         |       |
|Not partitioned|         |       |
+---------------+---------+-------+

Describing table: teststocksymbols
+---------------+---------+-------+
|col_name       |data_type|comment|
+---------------+---------+-------+
|symbol         |string   |       |
|name           |string   |       |
|               |         |       |
|# Partitioning |         |       |
|Not partitioned|         |       |
+---------------+---------+-------+

Describing table: teststocktransaction
+---------------+---------+-------+
|col_name       |data_type|comment|
+---------------+---------+-------+
|AAPL_price     |double   |       |
|AAPL_volume    |bigint   |       |
|AMZN_price     |dou